In [ ]:
#This notebook is to figure out how read in random patches to run a subset of them through N2V

In [18]:
from n2v.models import N2VConfig, N2V
import numpy as np
from csbdeep.utils import plot_history
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from matplotlib import pyplot as plt
import urllib
import os
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
import tifffile
import zipfile
import glob
from aicspylibczi import CziFile
from pathlib import Path
import random
import math
import time

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# %load_ext tensorboard
# %tensorboard --logdir Documents/DL_MBL_2022/N2V_models/n2v_3D_6channel/logs/train
# %tensorboard dev upload --logdir \
#     'Documents/DL_MBL_2022/N2V_models/n2v_3D_6channel/logs/train'

In [11]:
!pip install -U tensorboard 

  Using cached tensorboard-2.10.0-py3-none-any.whl (5.9 MB)
  Using cached protobuf-3.19.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.0
    Uninstalling tensorboard-2.9.0:
      Successfully uninstalled tensorboard-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0 which is incompatible.
tensorflow 2.4.1 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.4.1 requires grpcio~=1.32.0, but you have grpcio 1.42.0 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have n

In [16]:
!tensorboard dev upload --logdir \
    'Documents/DL_MBL_2022_1hot/N2V_models/n2v_3D_6channel/logs/'

2022-09-05 12:26:25.653200: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

Documents/DL_MBL_2022_1hot/N2V_models/n2v_3D_6channel/logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) ^C
Traceback (most recent call last):
  File "/home/rhoadss/miniconda3/envs/n2v/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/home/rhoadss/miniconda3/envs/n2v/lib/python3.7/s

In [17]:
yes

NameError: name 'yes' is not defined

In [ ]:
# # path = glob.glob('/mnt/shared/potatoshapes/shannonsdata/N2V_patches/*.tiff')
# len(path)

In [ ]:
# path = glob.glob('/mnt/shared/potatoshapes/shannonsdata/N2V_patches/*_validation_validation_train.tiff')
# len(path)

In [ ]:
# val_paths = path[:500]

In [ ]:
# for val_path in glob.glob("/mnt/shared/potatoshapes/shannonsdata/N2V_patches/*_validation_validation_train.tiff"):
#      new_path = val_path.replace("_validation_validation_train.tiff", "_validation.tiff")
#      os.rename(val_path, new_path)

In [ ]:
# for train_path in path:
#     new_path = train_path.replace(".tiff", "_train.tiff")
#     os.rename(train_path, new_path)

In [21]:
def get_subset_of_patches(patches):
    all_imgs = []
    for j in patches:
#         print(j)
        img = tifffile.imread(j)
#         print(img.shape)
        all_imgs.append(img)
#     print(all_imgs)
    return np.stack(all_imgs, axis=0)



In [22]:
path = glob.glob('/mnt/shared/potatoshapes/shannonsdata/N2V_patches/*train.tiff')

d_config = get_subset_of_patches(np.random.choice(path, 50, replace=False))

In [25]:
config = N2VConfig(d_config, 
                   unet_kern_size=3, 
                   train_steps_per_epoch=5,
                   train_epochs=20, 
                   train_loss='mse', 
                   batch_norm=True, 
                   train_batch_size=50, 
                   n2v_perc_pix=0.198, 
                   n2v_patch_shape=(8, 48, 48), 
                   n2v_manipulator='uniform_withCP', 
                   n2v_neighborhood_radius=5,
                   )
vars(config)

{'means': ['336.9163024902344',
  '264.4642730712891',
  '101.07105651855468',
  '200.98057983398436',
  '54.69622985839844',
  '60.3712890625'],
 'stds': ['291.56250705737597',
  '652.9299607709213',
  '312.59708041932987',
  '271.1678075876275',
  '268.4479170271138',
  '290.51899749822127'],
 'n_dim': 3,
 'axes': 'ZYXC',
 'n_channel_in': 6,
 'n_channel_out': 6,
 'unet_residual': False,
 'unet_n_depth': 2,
 'unet_kern_size': 3,
 'unet_n_first': 32,
 'unet_last_activation': 'linear',
 'unet_input_shape': (None, None, None, 6),
 'train_loss': 'mse',
 'train_epochs': 20,
 'train_steps_per_epoch': 5,
 'train_learning_rate': 0.0004,
 'train_batch_size': 50,
 'train_tensorboard': True,
 'train_checkpoint': 'weights_best.h5',
 'train_reduce_lr': {'factor': 0.5, 'patience': 10},
 'batch_norm': True,
 'n2v_perc_pix': 0.198,
 'n2v_patch_shape': (8, 48, 48),
 'n2v_manipulator': 'uniform_withCP',
 'n2v_neighborhood_radius': 5,
 'single_net_per_channel': True,
 'structN2Vmask': None,
 'probabilis

In [ ]:
config = N2VConfig(d_config, 
                   unet_kern_size=3, 
                   train_steps_per_epoch=5,
                   train_epochs=20, 
                   train_loss='mse', 
                   batch_norm=True, 
                   train_batch_size=50, 
                   n2v_perc_pix=0.198, 
                   n2v_patch_shape=(8, 48, 48), 
                   n2v_manipulator='uniform_withCP', 
                   n2v_neighborhood_radius=5,
                   )
del d_config
    
model_name = f'n2v_3D_6channel_{time.time()}'
basedir = 'models'
model = N2V(config=config, name=model_name, basedir=basedir)


path = glob.glob('/mnt/shared/potatoshapes/shannonsdata/N2V_patches/*train.tiff')
random.shuffle(path)

patchpergroup = 5000
numofgroup = math.ceil(len(path)/patchpergroup)
patch_subgroups = np.array_split(path, numofgroup)


path_val = glob.glob("/mnt/shared/potatoshapes/shannonsdata/N2V_patches/*_validation.tiff")
d_val = get_subset_of_patches(path_val)


for x,patches in enumerate(patch_subgroups[:1]):
    d_train = get_subset_of_patches(patches)
    history = model.train(d_train, d_val)
#     model.load_weights("weights_last.h5")


/home/rhoadss/miniconda3/envs/n2v/lib/python3.7/site-packages/n2v/models/n2v_standard.py:416: UserWarning: output path for model already exists, files may be overwritten: /home/rhoadss/Documents/DL_MBL_2022_1hot/models/n2v_3D_6channel
  'output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))
2022-09-04 02:37:47.438715: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-04 02:37:47.440027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-04 02:37:47.523799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 02:37:47.525399: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapa

36 blind-spots will be generated per training patch of size (8, 48, 48).


Preparing validation data: 100%|██████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1370.87it/s]
2022-09-04 02:38:58.683786: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-04 02:38:58.702988: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz


Epoch 1/20


2022-09-04 02:39:11.553685: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-09-04 02:39:12.952940: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-09-04 02:39:13.180158: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-09-04 02:39:15.392475: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


...

Epoch 1/20
5/5 [==============================] - 32s 7s/step - loss: 0.2742 - n2v_mse: 0.2742 - n2v_abs: 0.2449 - val_loss: 0.8503 - val_n2v_mse: 0.8354 - val_n2v_abs: 0.2939
Epoch 2/20
5/5 [==============================] - 32s 7s/step - loss: 1.1142 - n2v_mse: 1.1142 - n2v_abs: 0.2900 - val_loss: 0.8607 - val_n2v_mse: 0.8455 - val_n2v_abs: 0.2931
Epoch 3/20
5/5 [==============================] - 32s 7s/step - loss: 0.4335 - n2v_mse: 0.4335 - n2v_abs: 0.2500 - val_loss: 0.8661 - val_n2v_mse: 0.8509 - val_n2v_abs: 0.2915
Epoch 4/20
5/5 [==============================] - 32s 7s/step - loss: 0.5651 - n2v_mse: 0.5651 - n2v_abs: 0.2675 - val_loss: 0.8793 - val_n2v_mse: 0.8636 - val_n2v_abs: 0.2904
Epoch 5/20
5/5 [==============================] - 32s 7s/step - loss: 0.5508 - n2v_mse: 0.5508 - n2v_abs: 0.2854 - val_loss: 0.8859 - val_n2v_mse: 0.8701 - val_n2v_abs: 0.2910
Epoch 6/20
5/5 [==============================] - 32s 7s/step - loss: 0.5601 - n2v_mse: 0.5601 - n2v_abs: 0.2586 - val_l

Preparing validation data: 100%|██████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1450.44it/s]


Epoch 1/20
5/5 [==============================] - 32s 7s/step - loss: 1.0796 - n2v_mse: 1.0796 - n2v_abs: 0.2734 - val_loss: 0.7325 - val_n2v_mse: 0.7189 - val_n2v_abs: 0.2793
Epoch 2/20
5/5 [==============================] - 32s 7s/step - loss: 1.0144 - n2v_mse: 1.0144 - n2v_abs: 0.3041 - val_loss: 0.7318 - val_n2v_mse: 0.7183 - val_n2v_abs: 0.2794
Epoch 3/20
5/5 [==============================] - 32s 7s/step - loss: 0.4439 - n2v_mse: 0.4439 - n2v_abs: 0.2915 - val_loss: 0.7325 - val_n2v_mse: 0.7189 - val_n2v_abs: 0.2796
Epoch 4/20
5/5 [==============================] - 32s 7s/step - loss: 0.5701 - n2v_mse: 0.5701 - n2v_abs: 0.2792 - val_loss: 0.7341 - val_n2v_mse: 0.7205 - val_n2v_abs: 0.2796
Epoch 5/20
5/5 [==============================] - 32s 7s/step - loss: 2.2415 - n2v_mse: 2.2415 - n2v_abs: 0.2787 - val_loss: 0.7344 - val_n2v_mse: 0.7207 - val_n2v_abs: 0.2793
Epoch 6/20
5/5 [==============================] - 32s 7s/step - loss: 0.8179 - n2v_mse: 0.8179 - n2v_abs: 0.3006 - val_l

Preparing validation data: 100%|██████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1118.69it/s]


Epoch 1/20
5/5 [==============================] - ETA: 0s - loss: 0.4389 - n2v_mse: 0.4389 - n2v_abs: 0.2587